<a href="https://colab.research.google.com/github/antonpolishko/colab-notebooks-sink/blob/master/task-risk/CoronaWhy_Risk_Factors_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

task-risk 

https://coronawhy.slack.com/archives/C010NH92CH2/p1586792972453200


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import spacy
import re
from collections import Counter
import random

import scispacy
from tqdm import tqdm
import json
import io

import en_core_sci_lg
import en_ner_bc5cdr_md
import en_ner_bionlp13cg_md
from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker

import gensim
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.parsing.preprocessing import STOPWORDS

from sklearn.model_selection import train_test_split
from pprint import pprint


In [0]:
#default spacy model
nlp = spacy.load("en_core_web_lg")

In [0]:
dirs = ['biorxiv_medrxiv', 'comm_use_subset', 'custom_license', 'noncomm_use_subset']

In [0]:
root_path = 'C:/Users/MNgwenya/Documents/Data Science/Coronawhy/'  #'/kaggle/input/CO

In [0]:
main_root_path = "C:/Users/MNgwenya/Documents/Data Science/Kaggle/CORD-19-research-challenge (1)/"

In [0]:
canada_df = pd.read_excel(root_path + 'Public_COVID-19_Canada.xlsx') # 3409 records

In [0]:
covid_full = pd.read_csv(root_path + "covid_full_unabbreviated-method date of datetime.datetime objects.csv") # 29315 records

In [0]:
covid_sections = pd.read_json(root_path + "sections_export_1.json") # 29315 records

In [0]:
processed_df =  pd.read_csv(root_path + "tables_processed_03282020.csv") # 1025196 records

c:\users\mngwenya\pyspace\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,31,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
metadata_df = pd.read_csv(main_root_path + 'metadata.csv')

In [0]:
batch_1_processed_df = processed_df.iloc[:512598, :]

In [0]:
batch_2_processed_df = processed_df.iloc[512599:len(processed_df), :]

In [0]:
key_factors = ['asthma', 'tuberculosis',  'respiratory', 'pneumonia', 'hypertension', 'cardiovascular','infection', 'cerebrovascular', 'diabetes','hepatitis', 'chronic kidney', 'malignancy', 'immunodeficiency','coad']

In [0]:
import pickle as pic

In [0]:
 #pickled articles can be found https://drive.google.com/drive/folders/1jBBM0QwedzX8PfTvSK3ds6xKR6VWI4g0
pic_articles = open('covid19_articles', 'rb')

In [0]:
data = pic.load(pic_articles)

### Preprocess data

In [0]:
n_stop_words = [u'covid-19', u'SARS-CoV', u'HCoV-19', u'MERS-CoV', u'cov2', 'statistical']

In [0]:
def add_stop_words(sentences):
    for w in n_stop_words:
        sentences.vocab[w].is_stop = True        
    return sentences

In [0]:
def lemmantize_corpus(sentence):
    article = ''
    _sentence = sentence.lower()
    doc = nlp(_sentence)
    n_doc = add_stop_words(doc)
    for word in n_doc:
    # if it's not a stop word or punctuation mark, add it to our article!
        if not word.is_stop and not word.is_punct and len(word.text) != 1 and not word.like_num:
        # we add the lematized version of the word
            article += word.lemma_ + ' '
    
    return article

In [0]:
def preprocesstext(data):
    indeces = []
    cleaned_text = []
    
    #remove range [0: 75000] to preprocess all data which takes more time
    for i in (range(0, len(data[0 :75000]))):
        
        #Remove nonword characters and replace them by a single space
        fulltext = data.iloc[i, 3]
        index = data.iloc[i, 2]
        
        review = re.sub(r'\W', ' ', str(fulltext))
        #transform to lower cases
        review = str(fulltext).lower()
        #Remove single word charz such ' a ' 
        review = re.sub(r'\s+[a-z]\s+', ' ', review)
        #Remove space
        review = re.sub(r'\s+', ' ', review)
        #update the cleaned version
        lem_sentence = lemmantize_corpus(review)
        
        indeces.append(index)       
        cleaned_text.append(lem_sentence)   
        
    return (pd.DataFrame({'cleaned_text': cleaned_text, 'index':indeces})) 


In [0]:
cleaned_data = preprocesstext(data)

### Set up data for training word embedding model doc2vec

In [0]:
X_train, X_test = train_test_split(cleaned_data, test_size=0.25, random_state=42)

In [0]:
def process_corpus(data, tokens_only=False):
    for i in tqdm(range(0, len(data))):
        text = data.iloc[i, 0]
        index = data.iloc[i,1]
        
        if tokens_only:
            yield gensim.utils.simple_preprocess(text)
        else:
            # For training data, add tags
            yield TaggedDocument(gensim.utils.simple_preprocess(text), [index])

In [0]:
train_corpus = list(process_corpus(cleaned_data))

##### The current hyperparameters as they gave the best results, the distributed memory model seems to perform better more vector sizes hence the value of 100 was selected after multiple tests which did not yield good results. 
##### Min count was set to a higher value to avoid rare terms

In [0]:
import multiprocessing
cores = multiprocessing.cpu_count()

models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, window=8, min_count=19, vector_size=40, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, window=8, min_count=19, vector_size =100, workers=cores),
]
#model = Doc2Vec(dm=1, vector_size=40,min_count=10, epochs=100, workers=cores)

In [0]:
models[0].build_vocab(train_corpus)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

##### Model overfits the data when there are more 40 epochs, however this remains to be tested further with differing hyperparameters, in my case a higher values causes the model to overfit.

In [0]:
for model in tqdm(models):
    %time model.train(train_corpus, total_examples=model.corpus_count, epochs=40)

### Test word relations, this yields synonyms and word relations

In [0]:
model[0].wv.most_similar(['mortality'])

[('fatality', 0.6445885896682739),
 ('survival', 0.5999062061309814),
 ('incidence', 0.5373064279556274),
 ('lethality', 0.5291377305984497),
 ('death', 0.509006142616272),
 ('hospitalization', 0.4980815052986145),
 ('seropositivity', 0.4902401566505432),
 ('readmission', 0.46347740292549133),
 ('seroprevalence', 0.451874315738678),
 ('coinfection', 0.44803276658058167)]

### Find articles by subject

In [0]:
doc_x = 'animal'.split(' ')

#distributed model which is model[1] misbehaves with few words, hence bow model was used
test_inferred_vector = {}
test_inferred_vector = models[0].infer_vector(doc_x)
test_similar_docs = models[0].docvecs.most_similar([test_inferred_vector], topn=10)

### Testing similarities in papers

###### One can further retrain the model by excluding papers that are more directed at animal research
###### data can be filtered as show, remove records that are in id_to_exclude list or find similar records

###### <i style="color: red">data = data[~data['paper_ids'].isin(id_to_exclude)]</i>

In [0]:
id_to_exclude = []
for id in test_similar_docs:
    id_to_exclude.append(id[0])
    print(data[data['paper_ids'] == id[0]][['titles', 'paper_ids']].values[0])   

['Clinical Management of Captive Cheetahs'
 '095d29c85da8abc7d96e63e20f3f0051eb9ab373']
['Pathogen exposure in endangered island fox (Urocyon littoralis) populations: Implications for conservation management'
 '145b849ffc443529651d531b40b392078823f4ec']
['Genetic characterization of a new astrovirus detected in dogs suffering from diarrhoea'
 '7bb8b2a2d05a605169ac5614fef7ed49cd843dc8']
['Assessment of Virally Vectored Autoimmunity as a Biocontrol Strategy for Cane Toads'
 '8ba5919b2b4c229c9c1685852cf19a1d727ac220']
['Potential for Introduction of Bat-Borne Zoonotic Viruses into the EU: A Review'
 '38e37fa66e4acf2931dbc37ac44d66cce82bae55']
['Synchronized shift of oral, faecal and urinary microbiotas in bats and natural infection dynamics during seasonal reproduction'
 '02aabb0b8eb8a3f3692e162442bbed19c5f915b6']
['Peptide presentation by bat MHC class I provides new insight into the antiviral immunity of bats'
 '0d5f9dd2c1626d1d3a4228b838fc0bc666363a3c']
['The Ubiquitin Proteasome Syste

### Find Related Articles

In [0]:
#randomly pick up a word
def get_documents(doc, nth_records):
    
    similar = []
    indeces = []
    percentages = []
    n_model = []
    for model in models:
        
        inferred_vector = model.infer_vector(doc)
        similar_docs = model.docvecs.most_similar([inferred_vector], topn= nth_records)
        desc = []
        for index, value in (similar_docs):
            perc = round((value * 100), 2)
            indeces.append(index)
            percentages.append(perc)
            n_model.append(model)
                        
    return pd.DataFrame({'index':indeces, 'percentage': percentages, 'model': n_model })

##### Search for a related document in the corpus

In [0]:
doc = '''readmissions of patient'''.split(' ')
mydocs = get_documents(doc, 10)

In [0]:
mydocs

,index,percentage,model
0,8eb99586e9103599ab2f58cc33ca94cd0b60ae53,82.62,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
1,75435342678d143b3fea48a288d1baa113d28a42,82.01,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
2,2e3c6d9e2a2a8d2d9b43d5ae7b882c9a82557274,81.04,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
3,3078165241b24fed34596efa9f563661f1caa00c,80.28,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
4,196d55353760e8e1ddbc2e2cce92f806351b747c,79.85,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
5,88c599b1de8a17cd519f9f72aaaea28392f697f0,79.66,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
6,7ec733f3684d6294483b03df183878ad777a78d9,79.59,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
7,210b30ec755f5d8b0622c78a8e570290293bf49f,79.51,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
8,1ac070de6fe30924deb4ea2b0b8259884bde7121,79.39,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"
9,2ed23e6fe433d2ce14436eaa68a9ab6240a00500,79.12,"Doc2Vec(dm/m,d100,n5,w8,mc19,s0.001,t12)"


##### Find articles only in specified sections, we use this filter because the model was trained with all the sections of the paper, however in order to find comorbidities one would have to find a related section for instance the methodology section would talk about the characteristics of the patients. Also another problem is that the sections do not follow the same naming structure which makes it difficult to focus on a specific section hence in this notebook also search for related section 

In [0]:
def get_related_sections(target_section):
    
    target = nlp(target_section)
    _targets = []
    _sections = []
    _sims = []
    _paperids = []
    
    #only filter sections from the related documents
    for i in (range(0, len(mydocs))):
        
        #filter out introduction and background sections 
        sections = (data[(data['paper_ids'] == mydocs.iloc[i, 0]) & (~data['sections'].str.lower().str.contains('introduction')) & (~data['sections'].str.lower().str.contains('background'))][['sections']].drop_duplicates())
        
        for section in sections['sections']:
            section = section.lower()
            x_section = nlp(section)

            #for non-null sections find other sections related to that one
            if (len(section) > 0):
                similarity = target.similarity(x_section)
            else:
                similarity = 0

            _paperids.append(mydocs.iloc[i, 0])
            _targets.append(target_section)
            _sections.append(section)
            _sims.append(similarity)

    return pd.DataFrame({'paperids': _paperids, 'target': _targets, 'section':_sections, 'similarity': _sims })

In [0]:
get_related_sections('comorbidities')

C:\Users\MNgwenya\AppData\Local\Programs\Python\Python37\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


,paperids,target,section,similarity
0,8eb99586e9103599ab2f58cc33ca94cd0b60ae53,comorbidities,sample collection and analysis,0.052703
1,8eb99586e9103599ab2f58cc33ca94cd0b60ae53,comorbidities,phenotypic and epidemiological typing,0.398466
2,8eb99586e9103599ab2f58cc33ca94cd0b60ae53,comorbidities,ecological analysis,0.173376
3,8eb99586e9103599ab2f58cc33ca94cd0b60ae53,comorbidities,nestedness,0.000000
4,8eb99586e9103599ab2f58cc33ca94cd0b60ae53,comorbidities,species distributions,0.196752
...,...,...,...,...
171,2ed23e6fe433d2ce14436eaa68a9ab6240a00500,comorbidities,broad and higher antiviral activity,0.215885
172,2ed23e6fe433d2ce14436eaa68a9ab6240a00500,comorbidities,"the ""outliers"" of the unconventional ifn subtypes",0.193731
173,2ed23e6fe433d2ce14436eaa68a9ab6240a00500,comorbidities,discussion,0.016364
174,2ed23e6fe433d2ce14436eaa68a9ab6240a00500,comorbidities,data availability,0.161810


### Identify diseases mentioned per paper

In [0]:
def print_diseases_by_section(section):
    diseases = []
    paperids = []
    
    sims_sections_df = get_related_sections(section)
    
    # only take section that have are more than 45% similary to the searched for section
    # so sections such as findings and results are more similar
    filtered_sections = sims_sections_df[sims_sections_df['similarity'] > 0.45].sort_values('similarity', ascending=False)
    
    for i in (range(0, len(filtered_sections) - 120)):
        doc_x = (data[(data['paper_ids'] == filtered_sections.iloc[i, 0]) & (data['sections'].str.lower()== filtered_sections.iloc[i, 2])][['fulltext']].values)
        
        if (len(doc_x) > 0):
            perc = filtered_sections.iloc[i, 3] * 100
            #this simply shows sections with matching percentages
            print(str("{:.2f}".format(round(perc, 2))) + '% Section >>>', filtered_sections.iloc[i, 2])
            
            #display output and return disease mentioned per paperid
            output = display_entities(en_ner_bc5cdr_md, doc_x[0][0])
            
            diseases.append(output)
            paperids.append(sims_sections_df.iloc[i, 0])
    return pd.DataFrame({'diseases': diseases, 'paperids': paperids})

In [0]:
def display_entities(model, document):
    nlp = model.load()
    _document = document.lower()
    doc = nlp(_document)
    colors = {
    'CHEMICAL': '#ffffff',
    'DISEASE': '#b9ebcc',
    }
    
    counter = 0
    for e in doc.ents:
        if (e.label_ == 'DISEASE'):
            counter += 1
            if (counter > 1):
                
                displacy.render(doc, jupyter=True, style='ent', options= {"colors": colors})
                entity_and_label = [(x.text, x.label_) for x in doc.ents if x.label_ == 'DISEASE']
        
                return entity_and_label 

In [0]:
section = 'comorbidities'
output = print_diseases_by_section(section)

C:\Users\MNgwenya\AppData\Local\Programs\Python\Python37\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


### Get most common diseases mentioned together

### Get comorbidity associations e.g. (linked to/related to/associated with)

### Mortality rates mentioned with diseases (higher/lower/%)

### Comparisons in relation to comorbidities e.g. (more than/greater than/less than)

### Prevalence of comorbidities in COPD (chronic obstructive pulmonary disease)

### Save models for later use

In [0]:
model_path = "C:\\covidwhy.d2v"
model.save(model_path)

In [0]:
model_path = "C:\\covidwhy.d2v"
model = Doc2Vec.load(model_path)